In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset ,load_metric
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import pandas as pd
from datasets import Dataset
from datasets import load_dataset
import torch.nn.utils.prune as prune
from transformers import TrainingArguments,Trainer
import sacrebleu

2024-05-13 06:55:12.976746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 06:55:12.976808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 06:55:12.978496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
pip install sacrebleu

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the tokenizer and model
teacher_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)
student_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [5]:
# Ensure the teacher model does not track gradients
for param in teacher_model.parameters():
    param.requires_grad = False

In [6]:
teacher_model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [7]:

teacher_model.cuda()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [8]:
# Load WMT 2014 English-German dataset
dataset = load_dataset("wmt14", "de-en",split = 'train')
test_dataset = load_dataset('wmt14',"de-en",split = 'test')

val_dataset = load_dataset('wmt14',"de-en",split = 'validation')

In [9]:
sampled_dataset = dataset.shuffle(seed=42).select(range(int(0.001 * len(dataset))))
test_sample_dataset = test_dataset.shuffle(seed = 42).select(range(int(0.01*len(test_dataset))))

val_sample_dataset = val_dataset.shuffle(seed = 42).select(range(int(0.01*len(val_dataset))))

In [10]:
# Function to preprocess the data
def preprocess_function(examples):
    # Extracting German and English texts from the 'translation' dictionary
    inputs = [ex['de'] for ex in examples['translation']]
    targets = [ex['en'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding='max_length')

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


In [11]:
# Tokenize the dataset
tokenized_datasets = sampled_dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)
test_tokenized_datasets = test_sample_dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

val_tokenized_datasets = val_sample_dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

In [12]:
# # Define the custom trainer
# class DistillationTrainer(Trainer):
#     def __init__(self, teacher_model, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.teacher_model = teacher_model

#     def compute_loss(self, model, inputs, return_outputs=False):
#         outputs_student = model(**inputs)
#         with torch.no_grad():
#             inputs["decoder_input_ids"] = inputs["labels"]
#             outputs_teacher = self.teacher_model(**inputs)

#         loss_fn = torch.nn.MSELoss()
#         distillation_loss = loss_fn(outputs_student.logits, outputs_teacher.logits)

#         if return_outputs:
#             return (distillation_loss, outputs_student)
#         return distillation_loss

In [13]:
# Define the custom trainer
class DistillationTrainer(Trainer):
    def __init__(self, teacher_model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False):
        # Move all inputs to the correct device
        inputs = {k: v.to(self.args.device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
        
        outputs_student = model(**inputs)
        with torch.no_grad():
            # The teacher also needs labels for the forward pass
            inputs["decoder_input_ids"] = inputs.get("labels", inputs.get("decoder_input_ids"))
            outputs_teacher = self.teacher_model(**inputs)

        loss_fn = torch.nn.MSELoss()
        distillation_loss = loss_fn(outputs_student.logits, outputs_teacher.logits)

        if return_outputs:
            return (distillation_loss, outputs_student)
        return distillation_loss

In [14]:
import wandb
wandb.init(mode="disabled")

In [15]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=2,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=1,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     fp16 = True,  # enable mixed precision it help speed up training
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     logging_steps=10,
#     evaluation_strategy="epoch",
# #     evaluation_strategy="no",
# #     evaluation_strategy="steps",
#     save_strategy="epoch",
# #     load_best_model_at_end=True,
# )

In [17]:
# Initialize and start the trainer
trainer = DistillationTrainer(
    teacher_model=teacher_model,
    model=student_model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,No log,5.743519


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=70, training_loss=8.821241106305804, metrics={'train_runtime': 1462.6909, 'train_samples_per_second': 3.082, 'train_steps_per_second': 0.048, 'total_flos': 2731619332915200.0, 'train_loss': 8.821241106305804, 'epoch': 0.99})

In [25]:
def generate_translation(batch):
    # Assuming batch['translation'] is a list of dictionaries
    german_sentences = [item['de'] for item in batch['translation']]
    english_sentences = [item['en'] for item in batch['translation']]

    # Tokenize the German sentences
    inputs = tokenizer(german_sentences, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    inputs = {key: val.to(student_model.device) for key, val in inputs.items()}

    # Generate outputs
    outputs = student_model.generate(**inputs, max_length=512, num_beams=5)

    # Decode the outputs to human-readable translations
    translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return {"pred_translation": translations}

In [26]:
results =test_dataset.map(generate_translation, batched=True, batch_size=16)

Map:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [27]:
# Extract the translations and references
translations = [result['pred_translation'] for result in results]
references = [[ref['en']] for ref in results['translation']]

In [28]:
# Compute BLEU score using sacrebleu
bleu = sacrebleu.corpus_bleu(translations, references)
print(f"BLEU Score: {bleu.score}")

BLEU Score: 7.223943354597204
